In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import time

In [6]:
#Loading the webpage

url = 'https://stats.espncricinfo.com/ci/engine/stats/index.html?class=2;filter=advanced;orderby=runs;template=results;type=batting'
header = {'User Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'}
r = requests.get(url)
soup = BeautifulSoup(r.content,'lxml')
contentTable  = soup.find('table', { "class" : "engineTable"})#
print(contentTable)
print(soup.prettify())

<table class="engineTable" style="margin:0px;">
<tr class="data2">
<td class="left">
<b>View</b> overall figures
[<a href="javascript:void(0)" onmouseout="menuLayers.hide()" onmouseover="menuLayers.show('engine-dd-view', event); window.status='change view'; return true">change view</a>]

  </td>
</tr>
<tr class="data2">
<td class="left">
<b>Ordered by</b> runs scored (descending)

  </td>
</tr></table>
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<!-- hostname: web06, edition-view: , country: unknown, cluster: www, created: 2022-05-04 18:11:22 -->
<html xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/">
 <head>
  <script type="text/javascript">
   var _sf_startpt=(new Date()).getTime()
  </script>
  <meta content="ZxdgH3XglRg0Bsy-Ho2RnO3EE4nRs53FloLS6fkt_nc" name="google-site-verification"/>
  <meta content="width=device-width, ini

In [7]:
#Selecting the data table from the webpage

Player_table = soup.find_all("table",class_ = 'engineTable')[2]
#print(Player_table)

In [8]:
#Creating a template to extract data from the table in webpage

for name in Player_table.find_all('tbody'):
    rows = name.find_all('tr')
    for row in rows:
        player = row.find('td',class_ = 'left').text
        span = row.find_all('td')[1].text
        mat = row.find_all('td')[2].text
        inns = row.find_all('td')[3].text
        NO = row.find_all('td')[4].text
        runs = row.find_all('td')[5].text
        HS = row.find_all('td')[6].text
        Ave = row.find_all('td')[7].text
        BF = row.find_all('td')[8].text
        SR = row.find_all('td')[9].text
        Hundreds = row.find_all('td')[10].text
        Fifties = row.find_all('td')[11].text
        Zero = row.find_all('td')[12].text
        
        #Storing it in a dictionary
        Cricketers = {'Player':player,"Span":span,"Matches":mat,"Innings":inns,"NO":NO,"Runs":runs,"HS":HS,
                     "AVG":Ave,"Balls Faced":BF,"Strike Rate":SR,"100":Hundreds,"50":Fifties,"0":Zero}
        #print(Cricketers)


In [9]:
#Assigning an empty list to store all dictionaries of players

Cricketers_List = []

#Automating scrapping through different pages[the code has only first 2 page loop] by using a for loop and an f string edit in the link.

for page in range(1,56):
    source = requests.get(f'https://stats.espncricinfo.com/ci/engine/stats/index.html?class=2;filter=advanced;orderby=runs;page={page};template=results;type=batting')
    soup = BeautifulSoup(source.content,'lxml')
    Player_table = soup.find_all("table",class_ = 'engineTable')[2]
    
    #Code from the above cell which builds table structure
    
    for name in Player_table.find_all('tbody'):
        rows = name.find_all('tr')
        for row in rows:
            player = row.find('td',class_ = 'left').text
            span = row.find_all('td')[1].text
            mat = row.find_all('td')[2].text
            inns = row.find_all('td')[3].text
            NO = row.find_all('td')[4].text
            runs = row.find_all('td')[5].text
            HS = row.find_all('td')[6].text
            Ave = row.find_all('td')[7].text
            BF = row.find_all('td')[8].text
            SR = row.find_all('td')[9].text
            Hundreds = row.find_all('td')[10].text
            Fifties = row.find_all('td')[11].text
            Zero = row.find_all('td')[12].text

            Cricketers = {'Player':player,"Span":span,"Matches":mat,"Innings":inns,"NO":NO,"Runs":runs,"HS":HS,
                         "AVG":Ave,"Balls Faced":BF,"Strike Rate":SR,"100":Hundreds,"50":Fifties,"0":Zero}
            Cricketers_List.append(Cricketers)
    
           # print('Saving: ', Cricketers['Player'])

          #  time.sleep(3)

In [10]:
#Converting to DataFrame using pandas [sorted by number of runs by default]

df = pd.DataFrame(Cricketers_List)
df

,Player,Span,Matches,Innings,NO,Runs,HS,AVG,Balls Faced,Strike Rate,100,50,0
0,SR Tendulkar (INDIA),1989-2012,463,452,41,18426,200*,44.83,21368,86.23,49,96,20
1,KC Sangakkara (Asia/ICC/SL),2000-2015,404,380,41,14234,169,41.98,18048,78.86,25,93,15
2,RT Ponting (AUS/ICC),1995-2012,375,365,39,13704,164,42.03,17046,80.39,30,82,20
3,ST Jayasuriya (Asia/SL),1989-2011,445,433,18,13430,189,32.36,14725,91.20,28,68,34
4,DPMD Jayawardene (Asia/SL),1998-2015,448,418,39,12650,144,33.37,16020,78.96,19,77,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734,M Watkinson (ENG),1996-1996,1,-,-,-,-,-,-,-,-,-,-
2735,S Weerakoon (SL),2012-2012,2,-,-,-,-,-,-,-,-,-,-
2736,LB Williams (SA),2021-2021,1,-,-,-,-,-,-,-,-,-,-
2737,Zahir Khan (AFG),2019-2019,1,-,-,-,-,-,-,-,-,-,-


In [11]:
#Saving it into CSV

df.to_csv("ODI Batsmen.csv")

In [21]:
# Change the column names.
df.set_axis(['Player','ODI_Span','ODI_Matches','ODI_Innings','ODI_Notouts','ODI_RunsMade','ODI_Highest-Score','ODI_Batting_Average','ODI_Balls-Faced','ODI_Batting_Strikerate','ODI_Hundereds','ODI_Fifties','ODI_Ducks'],axis=1,inplace=True)

In [13]:
df=df.replace('-', 0)

In [19]:
df

,Player,ODI_Span,ODI_Matches,ODI_Innings,ODI_Notouts,ODI_Runsmade,ODI_Highest-Score,ODI_Batting_Average,ODI_Balls-Faced,ODI_Batting_Strikerate,ODI_Hundereds,ODI_Fifties,ODI_Ducks
0,SR Tendulkar (INDIA),1989-2012,463,452,41,18426,200*,44.83,21368,86.23,49,96,20
1,KC Sangakkara (Asia/ICC/SL),2000-2015,404,380,41,14234,169,41.98,18048,78.86,25,93,15
2,RT Ponting (AUS/ICC),1995-2012,375,365,39,13704,164,42.03,17046,80.39,30,82,20
3,ST Jayasuriya (Asia/SL),1989-2011,445,433,18,13430,189,32.36,14725,91.20,28,68,34
4,DPMD Jayawardene (Asia/SL),1998-2015,448,418,39,12650,144,33.37,16020,78.96,19,77,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734,M Watkinson (ENG),1996-1996,1,0,0,0,0,0,0,0,0,0,0
2735,S Weerakoon (SL),2012-2012,2,0,0,0,0,0,0,0,0,0,0
2736,LB Williams (SA),2021-2021,1,0,0,0,0,0,0,0,0,0,0
2737,Zahir Khan (AFG),2019-2019,1,0,0,0,0,0,0,0,0,0,0


In [23]:
#'Player','ODI_Span','ODI_Matches','ODI_Innings','ODI_Not-Outs','ODI_Runs',
#'ODI_Highest-Score','
#ODI_Avg',
#'ODI_Balls-Faced',#
#'ODI_Strike-Rate','ODI_Hundereds','ODI_Fifties','ODI_Ducks'

df['ODI_Matches']=df['ODI_Matches'].astype(int)
df['ODI_Innings']=df['ODI_Innings'].astype(int)
df['ODI_Notouts']=df['ODI_Notouts'].astype(int)
df['ODI_RunsMade']=df['ODI_RunsMade'].astype(int)
df['ODI_Balls-Faced']=df['ODI_Balls-Faced'].astype(int)
df['ODI_Hundereds']=df['ODI_Hundereds'].astype(int)
df['ODI_Fifties']=df['ODI_Fifties'].astype(int)
df['ODI_Ducks']=df['ODI_Ducks'].astype(int)
df["ODI_Batting_Average"] = pd.to_numeric(df['ODI_Batting_Average'], errors='coerce')
df["ODI_Batting_Strikerate"] = pd.to_numeric(df['ODI_Batting_Strikerate'], errors='coerce')
df.dtypes


Player                     object
ODI_Span                   object
ODI_Matches                 int32
ODI_Innings                 int32
ODI_Notouts                 int32
ODI_RunsMade                int32
ODI_Highest-Score          object
ODI_Batting_Average       float64
ODI_Balls-Faced             int32
ODI_Batting_Strikerate    float64
ODI_Hundereds               int32
ODI_Fifties                 int32
ODI_Ducks                   int32
dtype: object

In [24]:
df

,Player,ODI_Span,ODI_Matches,ODI_Innings,ODI_Notouts,ODI_RunsMade,ODI_Highest-Score,ODI_Batting_Average,ODI_Balls-Faced,ODI_Batting_Strikerate,ODI_Hundereds,ODI_Fifties,ODI_Ducks
0,SR Tendulkar (INDIA),1989-2012,463,452,41,18426,200*,44.83,21368,86.23,49,96,20
1,KC Sangakkara (Asia/ICC/SL),2000-2015,404,380,41,14234,169,41.98,18048,78.86,25,93,15
2,RT Ponting (AUS/ICC),1995-2012,375,365,39,13704,164,42.03,17046,80.39,30,82,20
3,ST Jayasuriya (Asia/SL),1989-2011,445,433,18,13430,189,32.36,14725,91.20,28,68,34
4,DPMD Jayawardene (Asia/SL),1998-2015,448,418,39,12650,144,33.37,16020,78.96,19,77,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734,M Watkinson (ENG),1996-1996,1,0,0,0,0,0.00,0,0.00,0,0,0
2735,S Weerakoon (SL),2012-2012,2,0,0,0,0,0.00,0,0.00,0,0,0
2736,LB Williams (SA),2021-2021,1,0,0,0,0,0.00,0,0.00,0,0,0
2737,Zahir Khan (AFG),2019-2019,1,0,0,0,0,0.00,0,0.00,0,0,0


In [25]:
df.to_excel('ODI Batsmenv1.xlsx')

In [3]:
#exporting the file as not running the complete code again
df_odi= pd.read_excel('ODI Batsmenv1.xlsx')

In [4]:
df_odi

,Unnamed: 0,Player,ODI_Span,ODI_Matches,ODI_Innings,ODI_Notouts,ODI_RunsMade,ODI_Highest-Score,ODI_Batting_Average,ODI_Balls-Faced,ODI_Batting_Strikerate,ODI_Hundereds,ODI_Fifties,ODI_Ducks
0,0,SR Tendulkar (INDIA),1989-2012,463,452,41,18426,200*,44.83,21368,86.23,49,96,20
1,1,KC Sangakkara (Asia/ICC/SL),2000-2015,404,380,41,14234,169,41.98,18048,78.86,25,93,15
2,2,RT Ponting (AUS/ICC),1995-2012,375,365,39,13704,164,42.03,17046,80.39,30,82,20
3,3,ST Jayasuriya (Asia/SL),1989-2011,445,433,18,13430,189,32.36,14725,91.20,28,68,34
4,4,DPMD Jayawardene (Asia/SL),1998-2015,448,418,39,12650,144,33.37,16020,78.96,19,77,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734,2734,M Watkinson (ENG),1996-1996,1,0,0,0,0,0.00,0,0.00,0,0,0
2735,2735,S Weerakoon (SL),2012-2012,2,0,0,0,0,0.00,0,0.00,0,0,0
2736,2736,LB Williams (SA),2021-2021,1,0,0,0,0,0.00,0,0.00,0,0,0
2737,2737,Zahir Khan (AFG),2019-2019,1,0,0,0,0,0.00,0,0.00,0,0,0


In [5]:
#dropping the unnamed index coming
df_odi=df_odi.drop(['Unnamed: 0'],axis=1)
df_odi

,Player,ODI_Span,ODI_Matches,ODI_Innings,ODI_Notouts,ODI_RunsMade,ODI_Highest-Score,ODI_Batting_Average,ODI_Balls-Faced,ODI_Batting_Strikerate,ODI_Hundereds,ODI_Fifties,ODI_Ducks
0,SR Tendulkar (INDIA),1989-2012,463,452,41,18426,200*,44.83,21368,86.23,49,96,20
1,KC Sangakkara (Asia/ICC/SL),2000-2015,404,380,41,14234,169,41.98,18048,78.86,25,93,15
2,RT Ponting (AUS/ICC),1995-2012,375,365,39,13704,164,42.03,17046,80.39,30,82,20
3,ST Jayasuriya (Asia/SL),1989-2011,445,433,18,13430,189,32.36,14725,91.20,28,68,34
4,DPMD Jayawardene (Asia/SL),1998-2015,448,418,39,12650,144,33.37,16020,78.96,19,77,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734,M Watkinson (ENG),1996-1996,1,0,0,0,0,0.00,0,0.00,0,0,0
2735,S Weerakoon (SL),2012-2012,2,0,0,0,0,0.00,0,0.00,0,0,0
2736,LB Williams (SA),2021-2021,1,0,0,0,0,0.00,0,0.00,0,0,0
2737,Zahir Khan (AFG),2019-2019,1,0,0,0,0,0.00,0,0.00,0,0,0


In [6]:
#As checked we are finding 2 major errors in player column.. 1st is some player are having name+(numericvalue)+(countryname)
#2nd having ICC/World/Asia/Afr along with country name
#1st removing (numeric data)
df_odi['Player']=df_odi['Player'].str.replace(r"\(.1?\)","")
df_odi['Player']=df_odi['Player'].str.replace(r"\(.2?\)","")
df_odi['Player']=df_odi['Player'].str.replace(r"\(.3?\)","")

<ipython-input-6-1f1b73470154>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_odi['Player']=df_odi['Player'].str.replace(r"\(.1?\)","")
<ipython-input-6-1f1b73470154>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_odi['Player']=df_odi['Player'].str.replace(r"\(.2?\)","")
<ipython-input-6-1f1b73470154>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df_odi['Player']=df_odi['Player'].str.replace(r"\(.3?\)","")


In [7]:
#2nd replacing ICC/World/Asia/Afr from the country name

df_odi= df_odi.replace('ICC/', '', regex=True)
df_odi= df_odi.replace('/ICC', '', regex=True)
df_odi= df_odi.replace('/Afr', '', regex=True)
df_odi= df_odi.replace('Afr/', '', regex=True)
df_odi= df_odi.replace('/Asia', '', regex=True)  
df_odi= df_odi.replace('Asia/', '', regex=True)  
df_odi= df_odi.replace('/World', '', regex=True)    


In [8]:
df_odi

,Player,ODI_Span,ODI_Matches,ODI_Innings,ODI_Notouts,ODI_RunsMade,ODI_Highest-Score,ODI_Batting_Average,ODI_Balls-Faced,ODI_Batting_Strikerate,ODI_Hundereds,ODI_Fifties,ODI_Ducks
0,SR Tendulkar (INDIA),1989-2012,463,452,41,18426,200*,44.83,21368,86.23,49,96,20
1,KC Sangakkara (SL),2000-2015,404,380,41,14234,169,41.98,18048,78.86,25,93,15
2,RT Ponting (AUS),1995-2012,375,365,39,13704,164,42.03,17046,80.39,30,82,20
3,ST Jayasuriya (SL),1989-2011,445,433,18,13430,189,32.36,14725,91.20,28,68,34
4,DPMD Jayawardene (SL),1998-2015,448,418,39,12650,144,33.37,16020,78.96,19,77,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734,M Watkinson (ENG),1996-1996,1,0,0,0,0,0.00,0,0.00,0,0,0
2735,S Weerakoon (SL),2012-2012,2,0,0,0,0,0.00,0,0.00,0,0,0
2736,LB Williams (SA),2021-2021,1,0,0,0,0,0.00,0,0.00,0,0,0
2737,Zahir Khan (AFG),2019-2019,1,0,0,0,0,0.00,0,0.00,0,0,0


In [9]:
df_odi.to_excel('ODI Batsmenv3.xlsx')